<a href="https://colab.research.google.com/github/llaygh/T5_camp_Amany/blob/main/Amany_YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
# prompt: library opencv

!pip install opencv-python


In [ ]:
!pip install ultralytics
!pip install opencv-p

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 17.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement opencv-p (from versions: none)
ERROR: No matching distribution found for opencv-p


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
from ultralytics import YOLO
model = YOLO('YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
import cv2
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')

In [ ]:
if not cap.isOpened():
  print('not successfult')
else:
  print('video opend successfully')

video opend successfully


## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
# Get video information using cv2
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Video's height
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))    # Video's width
fps = cap.get(cv2.CAP_PROP_FPS)                   # Video's frames per second (FPS)

# Print video information
print(f'Video Height: {height}')
print(f'Video Width: {width}')
print(f'Video FPS: {fps}')


Video Height: 1080
Video Width: 1920
Video FPS: 29.97002997002997


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
# Prepare Video Writer to Store the Output
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for output video
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (width, height))  # Output video file

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
# working with coordinates took lots of time ending with bad results unfortantly
# in addition to the run it each time i run the code to see the cordinates it takes time sometime the session crash

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
from google.colab.patches import cv2_imshow

count_left = 0
count_right = 0
count_top = 0
count_bottom = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference
    results = model(frame)

    # Extract detection results
    detections = results[0].boxes  # Get the boxes directly

    if detections is not None and len(detections) > 0:
        for box in detections:
            x1, y1, x2, y2 = box.xyxy[0]  # Get the bounding box coordinates
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))  # Convert to integers
            conf = box.conf.item()  # Get confidence score
            cls = int(box.cls.item())  # Get class ID

            cv2.rectangle(frame, (0, 0), (800, 50 ), (0, 255, 0), 2)  # Top edge
            cv2.rectangle(frame, (0, height - 50), (width, height), (0, 255, 0), 2)  # Bottom edge
            cv2.rectangle(frame, (0, 0), (50, height), (0, 255, 0), 2)  # Left edge
            cv2.rectangle(frame, (width - 50, 0), (width, height), (0, 255, 0), 2)  # Right edge

            #cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'Car {cls}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            #print(f'Detected class: {cls}, Coordinates: {x1, y1, x2, y2}')  # Debug print


            # if top :
            #  count_top +=1

            # if bottom :
            #  count_bottom +=1

            # if left :
            #  count_left +=1

            # if right :
            #  count_right +=1


    # Display counts on the frame
    # cv2.putText('Amany Alghamdi')
    # cv2.putText(frame, f'Vehicles Leaving (top): {count_left}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 0)
    # cv2.putText(frame, f'Vehicles Joining (down): {count_right}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 0)
    # cv2.putText(frame, f'Vehicles Leaving (Left): {count_left}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 0)
    # cv2.putText(frame, f'Vehicles Joining (Right): {count_right}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 0)

    # Write the frame to the output video
    out.write(frame)

    # Show the frame
    cv2_imshow(frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



0: 384x640 3 buss, 91 cars, 2 trucks, 2 vans, 2700.2ms
Speed: 4.6ms preprocess, 2700.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
from google.colab import files
# Release resources
cap.release()
out.release()  # Release the video writer
cv2.destroyAllWindows()

files.download('output.avi')

# Print final counts
# print(f'Total Cars Leaving (Right): {count_right}')
# print(f'Total Cars Joining (Left): {count_left}')

## Save and Submit Your Work

Submit both the notebook and the output video